In [1]:
import pandas as pd
import numpy as np

In [2]:
artists = "artists.dat"
tags = "tags.txt"
user_artists = "user_artists.dat"
user_friends = "user_friends.dat"
user_taggedartists_timestamps = "user_taggedartists-timestamps.dat"
user_taggedartists = "user_taggedartists.dat"

In [3]:
total_table = pd.read_table(user_taggedartists)
user_artists_df = pd.read_table(user_artists)
tags_df = pd.read_table(tags,encoding = "utf-8")
user_friends_df = pd.read_table(user_friends)
artists_df = pd.read_table(artists)

In [4]:
user_artists_df.head()
artists_df.columns = ["artistID","name","url","pictureURL"]
total_table = pd.merge(user_artists_df,artists_df,left_on = "artistID",right_on = "artistID")
total_table["weight_a"] = total_table["weight"]
total_table["weight_a"][total_table["weight_a"] > 0] = 1

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
df = total_table

#### 第一种方法: Model-based recommendation system(Collaborative filtering)

In [6]:
unique_ids = sorted(set(df.artistID))
unique_users = sorted(set(df.userID))
artist_ix = {id:ix for (ix,id) in enumerate(unique_ids)}
user_ix = {id:ix for (ix,id) in enumerate(unique_users)}
df['artist_ix'] = df.artistID.map(artist_ix)
df['user_ix'] = df.userID.map(user_ix)
df.head()

,userID,artistID,weight,name,url,pictureURL,weight_a,artist_ix,user_ix
0,2,51,13883,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,1,45,0
1,4,51,228,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,1,45,2
2,27,51,85,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,1,45,24
3,28,51,10,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,1,45,25
4,62,51,528,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,1,45,59


In [7]:
def norm(values):
    return values/values.max()
df['freq'] = df.groupby('user_ix').weight.transform(norm) #均一化weight
df.head()

,userID,artistID,weight,name,url,pictureURL,weight_a,artist_ix,user_ix,freq
0,2,51,13883,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,1,45,0,1.000000
1,4,51,228,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,1,45,2,0.045756
2,27,51,85,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,1,45,24,0.208333
3,28,51,10,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,1,45,25,0.370370
4,62,51,528,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,1,45,59,0.076323


In [8]:
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
df2 = df
artist_name = df2.set_index('artistID')['name'].to_dict()

def svd(df):
    m = csc_matrix((df.freq,(df.user_ix,df.artist_ix)))
    u,s,vt = svds(m,k=20)
    pred = np.dot(np.dot(u,np.diag(s)),vt)
    return pred

In [9]:
def user_recommendation(userID,pred):
    recommended = np.argsort(pred[user_ix.get(userID),:])[-8:]
    recommendations_list = [artist_name.get(r) for r in recommended]
    return recommendations_list

In [10]:
#对于现有的用户，进行推荐
usedID = int(input("Enter a new userID (chose a number below 2100):"))
print("-------------------------")
pred = svd(df)
recommendation = user_recommendation(usedID,pred)
print("Recommendation:")
print("-------------------------")
print(recommendation)

Enter a new userID (chose a number below 2100):2100
-------------------------
Recommendation:
-------------------------
['Lulu Santos', 'Delta Goodrem', 'Shakira', 'Keane', 'The Beatles', 'Hird', 'Michael Jackson', 'Vanessa Hudgens']


#### SVD recommendation

In [36]:
list_all_recommendation = []
list_index = []
pred = svd(df)

for i in range(2100):
    try:
        recommendation = user_recommendation(i,pred)
        list_all_recommendation.append(recommendation)
        print(i)
        list_index.append(i)
    except:
        continue

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
161
162
163
164
166
168
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
198
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
240
241
242
243
244
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
273
274
275
277
278
279
280
281
282
283
284
285
286
288
289
290
291
292
293
294
295

In [55]:
len(list_all_recommendation[6])

8

In [78]:
len(list_all_recommendation)

1891

In [79]:
len(list_index)

1891

In [81]:
list_tuple = []
for i in range(len(list_index)):
    for j in range(len(list_all_recommendation[i])):
        list_tuple.append((list_all_recommendation[i][j],list_index[i]))

In [82]:
list_tuple

[('Fallulah', 2),
 ('Digital Summer', 2),
 ('Chris Rea', 2),
 ('Cock Robin', 2),
 ('Reaper', 2),
 ('Faithless', 2),
 ('Talk Talk', 2),
 ('Mindless Self Indulgence', 2),
 ('A Rocket to the Moon', 3),
 ('The Beatles', 3),
 ('The Beat', 3),
 ('blink-182', 3),
 ('Suede', 3),
 ('The Rolling Stones', 3),
 ('The Easybeats', 3),
 ('Lulu Santos', 3),
 ('Air', 4),
 ('New Order', 4),
 ('Chris Rea', 4),
 ('James Blunt', 4),
 ('De/Vision', 4),
 ('Michael Jackson', 4),
 ('The Boats', 4),
 ('Faithless', 4),
 ('Ленинград', 5),
 ('Band of Horses', 5),
 ('Travis', 5),
 ('New Order', 5),
 ('blink-182', 5),
 ('Scissor Sisters', 5),
 ('James Blunt', 5),
 ('The Boats', 5),
 ('Ashley Tisdale', 6),
 ('Lulu Santos', 6),
 ('Nick Carter', 6),
 ('Deep Forest', 6),
 ('Peter Tosh', 6),
 ('Whitney Houston', 6),
 ('Britney Spears', 6),
 ('Boss AC', 6),
 ('Katy Perry', 7),
 ('Leona Lewis', 7),
 ('Whitney Houston', 7),
 ('Peter Tosh', 7),
 ('Cock Robin', 7),
 ('Britney Spears', 7),
 ('Ana Carolina', 7),
 ('Boss AC', 7)

In [83]:
with open('list_tuple.txt','w') as f:
    for line in list_tuple:
        f.write(str(line) + "\n")

##### 第二种方法 User-based recommendation system

In [ ]:
def similarity_score(person1,person2):
    item1 = set(df[df.userID==person1].artistID)
    item2 = set(df[df.userID==person2].artistID)
    common_items = set.intersection(*[item1, item2]) #算两个用户间，共同关注艺术家
    if len(common_items)==0:
        return 0
    sum_of_eclidean_distance = []
    for item in common_items: 
        freq1= float(df[(df.userID==person1)&(df.artistID==item)].freq) 
        freq2= float(df[(df.userID==person2)&(df.artistID==item)].freq) 
        sum_of_eclidean_distance.append(pow(freq1 - freq2,2)) #算共同关注艺术家之间的欧氏距离，作为两个人之间相似度的基准 
    sum_of_eclidean_distances = sum(sum_of_eclidean_distance)
    return 1/(1+math.sqrt(sum_of_eclidean_distances))

In [ ]:
def most_similar_users(person,number_of_users):
    scores_ = {other_person:similarity_score(person,other_person) for other_person in set(df.userID) if other_person != person}
    scores = sorted(scores_.items(), key=lambda d: d[1],reverse=True) #算相近的人 
    return scores[0:number_of_users]

In [ ]:
def user_recommend(person):
    recommend=[]
    similar_users = [i for i, j in most_similar_users(person,3)]
    item1 = set(df[df.userID==person].artistID)
    for other_user in similar_users:
        item2_ = set(df[df.userID==other_user].artistID)
        common_items_ = list(set.intersection(*[item1, item2_]))
        top_ = df[df.userID==other_user].sort_values('freq',ascending=False).artistID.tolist()
        top_2 = filter(lambda x: x not in common_items_ ,top_)
        top_2 = list(set(top_).difference(set(common_items_)))
        recommend.extend(top_2[0:4])
    
    recommends = list(set(recommend))
    #recommendataions_list = [artist_name.get(r) for r in recommends]
    return recommends

In [ ]:
#对于现有的用户，进行推荐
import math
usedID = int(input("Enter a new userID (chose a number below 2100):"))
print("-------------------------")
recommendation = user_recommend(usedID)
print("Recommendation")
for i in recommendation:
    print(i)
    print(artist_name.get(i))

In [ ]:
df[df.userID == 1600].name

### 对于新用户

In [ ]:
userID = input("Enter a new userID (chose a number over 3000):")
userID = int(userID)

In [ ]:
def get_information(count):
    new_data = {} 
    while count>0:
        artistID = input("Chose artists you like:")
        freq = input("How many stars do you want to evaluate this artists(1~5 stars is the highest):")
        artistID = int(artistID)
        freq = float(freq)/5
        count = count -1 
        new_data.update({artistID:freq})
    return new_data

In [ ]:
new_data = get_information(3)

In [ ]:
newDF = pd.DataFrame()
newDF['userID'] = [i for i in [userID]*3]
newDF['artistID'] = [i for i in new_data.keys()]
newDF['freq'] = [i for i in new_data.values()]

In [ ]:
df = df.append(newDF,ignore_index=True)

In [ ]:
df

In [ ]:
import math
your_recommend = user_recommend(3002)
print("TOP RECOMMENDATIONS FOR YOU:\n")

for r in your_recommend:
    print(artist_name.get(r))

#### 评价系统

In [ ]:
artist_tags = pd.read_pickle("artist_tags.pickle")

In [ ]:
artist_tags

In [ ]:
df_tags = pd.merge(df,artist_tags,on = "artistID")

In [ ]:
artist_tags.tags_dealed.head()

In [ ]:
artist_tags = artist_tags.reset_index()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

tf = TfidfVectorizer(ngram_range = (1,3),stop_words = 'english')
dtm = tf.fit_transform(artist_tags.tags_dealed)

In [ ]:
#降维
from scipy.sparse.linalg import svds
u1,s1,vt1 = svds(dtm , k = 100)
u1.shape, s1.shape, vt1.shape

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
artist_similarities = pairwise_distances(u1)

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
artist_similarities

In [ ]:
df[df.userID == 1600].name

#### 对老用户1600进行评价
给老用户推荐的是这些 707 196 98 7 234 1515 207 917 1143 1048 186 475

In [ ]:
list_i = [707,196,98,7,234,1515,207,917,1143,1048,186,475]
sum_i = 0
average_user = 0
average_i = 0
for j in list_j:

    print(sum_i <= average_user)
    sum_i = 0
    for i in list_i:
        sum_i += artist_similarities[j,i]
        average_i = sum_i/len(list_i)
        average_user = sum(artist_similarities[j,])/len(artist_similarities[j,]) #55号艺术家
        
#sum_i/len(list_i)

In [ ]:
list_j

In [ ]:
list_j = list(df_tags[df_tags.userID == 1600]["artistID"])

In [ ]:
artist_similarities[55,1048]

In [ ]:
artist_similarities[1600].argsort()